# **DIMEMEX — Complete Project Pipeline Notebook**

*Multilingual Meme Translation & Hate Speech Analysis*

---

## **Members**

* **Bárbara** (Text)
* **Amanda** (Text + Description)
* **Juan David Nieto** (Text + Description + Image)
* **Luisa** (Image)

---

## **Project Overview**

This project analyzes whether **offensive or hate speech content in memes is preserved after translating the memes from Spanish to Portuguese**.

We work with the **DIMEMEX** dataset, which contains:

* Meme **text**
* Meme **description**
* Meme **image**
* Labels: *hate speech*, *inappropriate content*, *neither*

This project has **two main tasks**:

### **Task 1 — Translation Quality Evaluation**

Translate the Spanish text to Portuguese and evaluate translation quality using standard NLP metrics:

* **BLEURT**
* **BERTScore**
* **COMET-Kiwi**
* **chrF**

### **Task 2 — Hate Speech Detection (Multimodal Fine-Tuning)**

Fine-tune models to classify:

* Hate speech
* Inappropriate content
* Neither

We fine-tune under four input settings:

1. Text
2. Text + Description
3. Text + Description + Image (Multimodal)
4. Image

---

## **📌 Objectives**

1. Evaluate whether offensive content is **maintained or lost** during translation.
2. Compare performance of **original Spanish memes vs. translated Portuguese memes**.
3. Train and evaluate multimodal detectors to classify hate speech.
4. Analyze cases where the label changes across languages.
5. Perform **human qualitative analysis** on inconsistent cases.



# Imports

# 🛠️ 1. Install Dependencies

In [1]:
!pip install -q transformers accelerate datasets peft bitsandbytes torch torchvision pillow tqdm scikit-learn matplotlib evaluate
!pip install -q trl # Install TRL for SFTTrainer

In [2]:
import os, zipfile, json, warnings, torch, evaluate

import pandas as pd
import numpy as np
from PIL import Image

from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

from datasets import Dataset

# Hugging Face - Transformers (Models, Processors, Configs)
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    AutoModelForCausalLM,
    Idefics3ForConditionalGeneration,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    Trainer
)

# Hugging Face - PEFT (LoRA)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)

# Hugging Face - TRL (Training)
from trl import SFTTrainer

from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# 🗂️ 2. Upload Data

In [3]:
from google.colab import drive

# Montar Drive
drive.mount('/content/drive')

# === Caminhos dos CSVs (ajuste se preferir) ===
CSV_TRAIN = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/train/dados_espanhol.csv"
CSV_VAL   = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/validation/dados_espanhol.csv"
CSV_TEST  = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/test/dados_espanhol.csv"

# === Caminhos das pastas de imagens ===
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/train_images"
VAL_IMAGES_DIR   = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/validation_images"
TEST_IMAGES_DIR  = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/test_images"

# Carregar CSVs
df_train = pd.read_csv(CSV_TRAIN)
df_val   = pd.read_csv(CSV_VAL)
df_test  = pd.read_csv(CSV_TEST)

print("CSV train:", df_train.shape)
print("CSV val:", df_val.shape)
print("CSV test:", df_test.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV train: (2262, 4)
CSV val: (322, 4)
CSV test: (648, 4)


# ⚙️ 3. Main Configurations

In [4]:
# --- Model & Training Params ---
MODEL_ID = "HuggingFaceTB/SmolVLM-256M-Instruct"
BATCH_SIZE = 2
EPOCHS = 2
LR = 2e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Label distribution (string labels only) ---
print("\n📊 Quantidade de exemplos por label:")
print(df_train["label"].value_counts())


📊 Quantidade de exemplos por label:
label
hate speech              1404
inappropriate content     472
neither                   386
Name: count, dtype: int64


#📦 4. Prepare Train, Validation, and Test Datasets

In [5]:
# Os dados já estão divididos

df_train["image_path"] = df_train["image_path"].apply(lambda x: os.path.join(TRAIN_IMAGES_DIR, x))
df_val["image_path"]   = df_val["image_path"].apply(lambda x: os.path.join(VAL_IMAGES_DIR, x))
df_test["image_path"]  = df_test["image_path"].apply(lambda x: os.path.join(TEST_IMAGES_DIR, x))

ds_train = Dataset.from_pandas(df_train.reset_index(drop=True))
ds_val = Dataset.from_pandas(df_val.reset_index(drop=True))
ds_test = Dataset.from_pandas(df_test.reset_index(drop=True))

print(f"Train: {len(ds_train)} | Validation: {len(ds_val)} | Test: {len(ds_test)}")

Train: 2262 | Validation: 322 | Test: 648


#🧩 5. Processor and Pre-processing **(Image)**

In [7]:
# Se o processor não estiver definido anteriormente
if 'processor' not in locals():
    processor = AutoProcessor.from_pretrained(MODEL_ID)

def format_for_multimodal(example):
    # 1. Carregar imagem
    path = example["image_path"]
    try:
        if not os.path.exists(path):
            # Fallback imagem preta
            image = Image.new("RGB", (224, 224), "black")
        else:
            image = Image.open(path).convert("RGB")
    except Exception:
        image = Image.new("RGB", (224, 224), "black")

    # 2. Prompt fixo (apenas visual)
    prompt_text = "Analyze the meme image and classify it. What is the correct category?"

    # 3. Estrutura de mensagens (User + Image / Assistant)
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt_text}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": example['label']}
            ]
        }
    ]

    # Retornamos um dicionário com objetos python, não tensores ainda
    return {"images": image, "messages": messages}

# Aplicar o map
# Removemos as colunas antigas para o dataset ficar leve e limpo
original_cols = ds_train.column_names
ds_train = ds_train.map(format_for_multimodal, remove_columns=original_cols)
ds_val = ds_val.map(format_for_multimodal, remove_columns=original_cols)

print("Colunas atuais:", ds_train.column_names)
print("Exemplo 0 messages:", ds_train[0]['messages'])

Map:   0%|          | 0/2262 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

Colunas atuais: ['images', 'messages']
Exemplo 0 messages: [{'content': [{'text': None, 'type': 'image'}, {'text': 'Analyze the meme image and classify it. What is the correct category?', 'type': 'text'}], 'role': 'user'}, {'content': [{'text': 'hate speech', 'type': 'text'}], 'role': 'assistant'}]


# 🧠 6. Model + LoRA

In [11]:
# 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = Idefics3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

base_model = prepare_model_for_kbit_training(base_model)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # Target modules updated for Idefics3/SmolVLM
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

trainable params: 5,769,216 || all params: 262,254,144 || trainable%: 2.1999


# 🧮 7. Training Configuration

In [12]:
# Isso aqui é por causa das imagens
# 1. Definir o Data Collator (Necessário para corrigir o erro de dimensão)
class DataCollatorSmolVLM:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, examples):
        texts = []
        images = []

        for example in examples:
            # Converte a lista de mensagens em string formatada
            text = self.processor.apply_chat_template(example["messages"], add_generation_prompt=False)
            texts.append(text)
            images.append(example["images"])

        # O processor aqui lida com o padding e tensores de imagem automaticamente
        batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)

        # Configurar labels (mascarando o padding)
        labels = batch["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100
        batch["labels"] = labels

        return batch

# Instanciar o collator
data_collator = DataCollatorSmolVLM(processor)

# 2. Configurar Argumentos de Treino
training_args = TrainingArguments(
    output_dir="./SmolVLM_DIMEMEX_ImageOnly",
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    remove_unused_columns=False,
    report_to="none"
)

# 3. Instanciar o Trainer Padrão (não SFTTrainer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    data_collator=data_collator,
)

# 9. Train Model

In [ ]:
trainer.train()
print("Fine-tuning complete.")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


# 10. Loss Curves

In [ ]:
# Garante que o diretório de saída existe
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Acessa o histórico direto da memória
logs = trainer.state.log_history

train_steps = [x["step"] for x in logs if "loss" in x]
train_loss = [x["loss"] for x in logs if "loss" in x]

eval_steps = [x["step"] for x in logs if "eval_loss" in x]
eval_loss = [x["eval_loss"] for x in logs if "eval_loss" in x]

plt.figure(figsize=(10, 5))
plt.plot(train_steps, train_loss, label="Train Loss")
plt.plot(eval_steps, eval_loss, label="Eval Loss", marker='o')
plt.title("Training and Evaluation Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# --- SALVAR O GRÁFICO ---
plot_path = os.path.join(OUTPUT_DIR, "loss_curve.png")
plt.savefig(plot_path)
print(f"📉 Gráfico de Loss salvo em: {plot_path}")

plt.show()

###**Só pode rodar a célula abaixo quando estiver satisfeito com os resultado! Parão não ter vazamento de dados**

# ✅ 11. Final Evaluation and Metrics Summary

In [ ]:
# Use o modelo treinado
model = trainer.model
model.eval()

test_predictions = []
test_ground_truth = []
valid_labels = ["hate speech", "inappropriate content", "neither"]

print("🚀 Rodando avaliação final no conjunto de teste...")

# Loop pelo Test Set
for example in tqdm(ds_test):
    try:
        image = Image.open(example['image_path']).convert("RGB")
    except:
        continue

    prompt_text = "Analyze the meme image and classify it. What is the correct category?"

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt_text}
            ]
        }
    ]

    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=20, do_sample=False)

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Limpeza da resposta
    if "Assistant:" in generated_text:
        prediction = generated_text.split("Assistant:")[-1].strip()
    elif "assistant" in generated_text:
         prediction = generated_text.split("assistant")[-1].strip()
    else:
        prediction = generated_text.strip()

    test_predictions.append(prediction)
    test_ground_truth.append(example['label'])

In [ ]:
# --- Processamento e Salvamento dos Resultados ---

# 1. Limpeza e Filtro
cleaned_preds = []
for p in test_predictions:
    p_clean = p.lower().replace(".", "").strip()
    if p_clean in valid_labels:
        cleaned_preds.append(p_clean)
    else:
        cleaned_preds.append("unknown") # Marca erro de geração

# 2. Preparar listas finais
final_preds = []
final_gt = []
for p, g in zip(cleaned_preds, test_ground_truth):
    if p in valid_labels:
        final_preds.append(p)
        final_gt.append(g)

# 3. SALVAR PREDIÇÕES (CSV)
# Isso cria uma tabela com: Label Real | Predição | Acertou?
df_results = pd.DataFrame({
    "ground_truth": final_gt,
    "prediction": final_preds
})
df_results["correct"] = df_results["ground_truth"] == df_results["prediction"]
csv_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
df_results.to_csv(csv_path, index=False)
print(f"\n💾 Tabela de predições salva em: {csv_path}")

# 4. Calcular Métricas
report_str = classification_report(final_gt, final_preds, labels=valid_labels)
results = {
    "accuracy": accuracy_score(final_gt, final_preds),
    "f1_weighted": f1_score(final_gt, final_preds, average="weighted"),
    "precision_weighted": precision_score(final_gt, final_preds, average="weighted", zero_division=0),
    "recall_weighted": recall_score(final_gt, final_preds, average="weighted", zero_division=0)
}

# 5. SALVAR RELATÓRIO (TXT e JSON)
txt_path = os.path.join(OUTPUT_DIR, "classification_report.txt")
with open(txt_path, "w") as f:
    f.write(report_str)
    f.write("\n\nSummary Metrics:\n")
    f.write(json.dumps(results, indent=4))

json_path = os.path.join(OUTPUT_DIR, "metrics.json")
with open(json_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"📄 Relatórios salvos em: {txt_path} e {json_path}")

# Exibir na tela
print(f"\n{len(final_preds)} / {len(test_predictions)} predições válidas.")
print("\n📊 Resultados Finais:")
print(report_str)

# 💾 12. Save Final Model

In [ ]:
# Save the LoRA adapters
model.save_pretrained(OUTPUT_DIR)
# Save the processor
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Fine-tuning complete. Model saved to: {OUTPUT_DIR}")